In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime

In [20]:
train_data = pd.read_csv("02012009_29122017.csv")
test_data = pd.read_csv("02012018_31122018.csv")
train_data.head(5)
test_data.head(5)

,Date,Open Price,Close Price,High Price,Low Price,Volume
0,02-Jan-2018,2683.73,2695.81,2695.89,2682.36,1846463232
1,03-Jan-2018,2697.85,2713.06,2714.37,2697.77,2090595328
2,04-Jan-2018,2719.31,2723.99,2729.29,2719.07,2100767744
3,05-Jan-2018,2731.33,2743.15,2743.45,2727.92,1918869120
4,08-Jan-2018,2742.67,2747.71,2748.51,2737.60,1894823936


- 只留最高點和最低點
- 再加上一個'movement'
- Date 會影響計算所以這裡也砍掉

In [21]:
train_data = train_data.drop(['Date', 'High Price', 'Low Price', 'Volume'], axis=1)
test_data = test_data.drop(['Date', 'High Price', 'Low Price', 'Volume'], axis=1)

train_mask = train_data['Open Price']>train_data['Close Price']
test_mask = test_data['Open Price']>test_data['Close Price']

train_data.insert(2,'movement',1)
test_data.insert(2,'movement',1)

train_data.loc[train_mask, 'movement']=0
test_data.loc[test_mask, 'movement']=0

train_data['movement'] = train_data['movement'].shift(-1)
test_data['movement'] = test_data['movement'].shift(-1)

#train_data['Date'] = train_data['Date'].map(lambda x:time.mktime(datetime.datetime.strptime(x, "%d-%b-%Y").timetuple()))
#test_data['Date'] = test_data['Date'].map(lambda x:time.mktime(datetime.datetime.strptime(x, "%d-%b-%Y").timetuple()))

In [22]:
train_data = train_data.dropna()
test_data = test_data.dropna()

train_data['movement'] = train_data['movement'].astype('int_')
test_data['movement'] = test_data['movement'].astype('int_')

x_train = train_data.loc[:, train_data.columns != 'movement']
y_train = train_data[['movement']]
x_test = test_data.loc[:, test_data.columns != 'movement']
y_test = test_data[['movement']]
y_train.head()

,movement
0,0
1,1
2,0
3,1
4,0


In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

x_train_std = scaler.transform(x_train)
x_test_std = scaler.transform(x_test)

In [24]:
from sklearn.svm import SVC

In [25]:
from sklearn.metrics import accuracy_score

# Custom kernel function
def my_kernel(X, Y):
    return np.dot(X, Y.T)
  
penalty = 0.05

svm = SVC(C=penalty, kernel="linear", probability=True)
svm.fit(x_train, y_train['movement'].values)
  
acc_rate = accuracy_score(y_test['movement'].values, svm.predict(x_test)) * 100
print("Penalty = %.2f, Accuracy = %.2f %%" % (penalty, acc_rate))

Penalty = 0.05, Accuracy = 50.20 %


In [26]:
for i, sv in enumerate(zip(svm.support_, svm.support_vectors_)):
    if i < 5:
        print("Index: %s; Support vector: %s" % (sv[0], sv[1]))

Index: 0; Support vector: [902.99 931.8 ]
Index: 2; Support vector: [931.17 934.7 ]
Index: 4; Support vector: [905.73 909.73]
Index: 5; Support vector: [909.91 890.35]
Index: 7; Support vector: [869.79 871.79]


In [27]:
svm.n_support_

array([1027, 1029])

In [28]:
print("The labels by model prediction : %s" % svm.predict(x_test))
print("Real labels in the testing set : %s" % y_test['movement'].values)

The labels by model prediction : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Real labels in the testing set : [1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 0 0 0 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 1 1
 0 0 0 1 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 0
 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 0 0 0 1 0 1 1 1 1 0 1 1 1
 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 0 1 0 1 1
 1 1 1 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0 1 1 1 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 

In [29]:
for prob in svm.predict_proba(x_test):
    print(["%.2f" % p for p in prob] )

['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.49', '0.51']
['0.49', '0.51']
['0.48', '0.52']
['0.49', '0.51']
['0.48', '0.52']
['0.48', '0.52']
['0.49', '0.51']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.49', '0.51']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.47', '0.53']
['0.46', '0.54']
['0.50', '0.50']
['0.48', '0.52']
['0.46', '0.54']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.49', '0.51']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.49', '0.51']
['0.49', '0.51']
['0.47', '0.53']
['0.47', '0.53']
['0.47', '0.53']
['0.49', '0.51']
['0.49', '0.51']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.49', '0.51']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.48', '0.52']
['0.47', '0.53']
['0.47', '0.53']
['0.49', '0.51']
['0.47', '0.53

In [30]:
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.6, 
                    c = ['red', 'blue', 'lightgreen'][idx],
                    edgecolor='black',
                    marker=markers[idx], 
                    label=cl)

    # highlight test samples
    if test_idx:
        # plot all samples
        if not versiontuple(np.__version__) >= versiontuple('1.9.0'):
            X_test, y_test = X[list(test_idx), :], y[list(test_idx)]
            warnings.warn('Please update to NumPy 1.9.0 or newer')
        else:
            X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c='',
                    alpha=1.0,
                    edgecolor='black',
                    linewidths=1,
                    marker='o',
                    s=55, label='test set')

In [31]:
def plot_train_test(location, x, y):
  
    plt.subplot(1,2,location)

    plot_decision_regions(x, y, classifier=svm)
    if location == 1:
      plt.title("Training")
    elif location == 2:
      plt.title("Testing")
    plt.xlabel('Open')
    plt.ylabel('Close')
    plt.legend(loc='upper left')
    plt.tight_layout()

In [44]:
penalty = 0.05

svm = SVC(C=penalty, kernel="linear")
svm.fit(x_train, y_train['movement'].values)

acc_train = accuracy_score(y_train['movement'].values, svm.predict(x_train)) * 100
acc_test = accuracy_score(y_test['movement'].values, svm.predict(x_test)) * 100
print("Penalty = %.2f, Train accuracy = %.2f %%, Test accuracy = %.2f %%" % (penalty, acc_train, acc_test))

Penalty = 0.05, Train accuracy = 54.62 %, Test accuracy = 50.20 %


In [47]:
#plt.figure(figsize=(10,5))

#plot_train_test(1, x_train.values, y_train['movement'].values)
#plot_train_test(1, x_test.values, y_test['movement'].values)

#plt.show()

print("C: %.2f, Train Acc: %.2f %%, Test Acc: %.2f %%" % (penalty, acc_train, acc_test))
print("Kernel function : linear")

C: 0.05, Train Acc: 54.62 %, Test Acc: 50.20 %
Kernel function : linear
